In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay
from scipy import stats
import pandas as pd
import collections

In [2]:
genCollection = "u/lgiraldo/bps_ALLCCD_20220701"
butler = dB.Butler("/repo/main/", collections=genCollection)

In [4]:
CCD = 1
ptcDataset = butler.get("ptc", instrument="LSSTCam", detector=CCD)

# Exposure pairs list
exposure_pairs = np.array(sorted(ptcDataset.inputExpIdPairs['C00']))
exposure_first = exposure_pairs[:,0][:,0]

In [4]:
gainx = collections.deque([])
fluxx = collections.deque([])
rd_noisex = collections.deque([])
amp_namesx = collections.deque([])
detectorx = collections.deque([])


for k in range(180,189):
    for i in exposure_first:

        cp = butler.get("cpCovariances", instrument="LSSTCam", detector=k, exposure= i)
        for j in range(16):
            amp_namesx.append(list(cp.gain.keys())[j])
            gainx.append(list(cp.gain.values())[j])
            fluxx.append(list(cp.rawMeans.values())[j])
            rd_noisex.append(list(cp.noise.values())[j])
            detectorx.append(k)

In [5]:
df2 = pd.DataFrame(list(zip(detectorx, amp_namesx, gainx, rd_noisex, np.array(fluxx).T[0])),columns = ['Detector', 'Amplifier', 'Gain', 'Read_Noise', 'Flux'])
df2.to_feather("./cpCovariances_data19.ft")

In [7]:
for k in range(180,189):
    print(k)

180
181
182
183
184
185
186
187
188


In [3]:
detector1 = collections.deque([])
amplifier1 = collections.deque([])
gain1 = collections.deque([])
rd_noise1 = collections.deque([])
a001 = collections.deque([])
turnoff1 = collections.deque([])
k = 0

for i in range(189):
    ptcDataset = butler.get("ptc", instrument="LSSTCam", detector=i)
    amp_names1 = ptcDataset.ampNames
    for i in range(16):
        gain1.append(list(ptcDataset.gain.values())[i])
        rd_noise1.append(list(ptcDataset.noise.values())[i])
        turnoff1.append(list(ptcDataset.ptcTurnoff.values())[i])
        a001.append(np.array(list(ptcDataset.ptcFitPars.values()))[:,0][i])
        detector1.append(k)
    k+=1

df_PTC = pd.DataFrame(list(zip(detector1, amp_names1, gain1, rd_noise1, a001, turnoff1)),columns = ['Detector', 'Amplifier', 'Gain', 'Read_Noise', 'A00', 'TurnOff'])

In [5]:
df_PTC.to_csv("PTC_data.txt", index = False, sep=',')
df_PTC

,Detector,Amplifier,Gain,Read_Noise,A00,TurnOff
0,0,C00,1.602594,6.035249,-0.000002,74918.561200
1,0,C01,1.593339,5.937206,-0.000002,73830.511171
2,0,C02,1.590703,6.018622,-0.000002,72504.390664
3,0,C03,1.589087,6.080679,-0.000002,72801.819624
4,0,C04,1.583272,6.206142,-0.000002,73461.156208
...,...,...,...,...,...,...
3019,188,C13,1.718498,5.825182,-0.000002,62274.175580
3020,188,C14,1.726264,5.895201,-0.000002,58818.706056
3021,188,C15,1.711954,5.738058,-0.000002,59050.745682
3022,188,C16,1.697910,5.683215,-0.000002,57872.448281


In [24]:
import glob
import os
all_files = sorted(glob.glob("cpCovariances_data*.ft"))

In [25]:
all_files

['cpCovariances_data01.ft',
 'cpCovariances_data02.ft',
 'cpCovariances_data03.ft',
 'cpCovariances_data04.ft',
 'cpCovariances_data05.ft',
 'cpCovariances_data06.ft',
 'cpCovariances_data07.ft',
 'cpCovariances_data08.ft',
 'cpCovariances_data09.ft',
 'cpCovariances_data10.ft',
 'cpCovariances_data11.ft',
 'cpCovariances_data12.ft',
 'cpCovariances_data13.ft',
 'cpCovariances_data14.ft',
 'cpCovariances_data15.ft',
 'cpCovariances_data16.ft',
 'cpCovariances_data17.ft',
 'cpCovariances_data18.ft',
 'cpCovariances_data19.ft']

In [33]:
df_cpCov = pd.concat((pd.read_feather(f) for f in all_files), ignore_index=True)

In [49]:
df_cpCov.to_feather("cpCovariances_data_ALL.ft")

In [ ]:
df_cpCov = pd.read_feather("cpCovariances_data_ALL.ft")

In [48]:
df_cpCov[df_cpCov.Detector == 188]

,Detector,Amplifier,Gain,Read_Noise,Flux
1016704,188,C00,1.763129,3.395242,55.713691
1016705,188,C01,1.709736,3.788210,54.638449
1016706,188,C02,1.775803,3.431862,55.480985
1016707,188,C03,1.775853,3.454132,55.452376
1016708,188,C04,1.668705,3.469973,54.752178
...,...,...,...,...,...
1022107,188,C13,2.166906,3.435512,58013.246904
1022108,188,C14,2.179758,3.459857,57480.759964
1022109,188,C15,2.161555,3.415035,57704.512458
1022110,188,C16,2.151485,3.490749,57872.448281


In [37]:
det = [0 ,2 ,4 ,6 ,8 ,10,12,15,17,19,21,23,25,27,30,32,34,36,38,40,42,45]

In [50]:
det0 = df_cpCov[df_cpCov.Detector == det[0]]

In [65]:
nan_flux = df_cpCov.loc[df_cpCov.Flux.isnull()]
nan_flux.Detector.drop_duplicates()

389391     72
634680    117
Name: Detector, dtype: int64

In [58]:
np.where(exposure_first == 3021120600576)

(array([0]),)

In [59]:
exposure_first [0]

3021120600576

In [61]:
det0.loc[0]

Detector              0
Amplifier           C00
Gain           1.595276
Read_Noise     3.634567
Flux          47.787885
Name: 0, dtype: object